In [34]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [35]:
products_file = "./ProdottiKebab/ProdottiVecchi.json"
orders_dir = "./OrdiniKebab"

In [36]:
products_df = pd.read_json(products_file)
products_df

,name,category,price,ingredients
0,marinara,pizza,3.8,"[pomodoro, origano, aglio, olio]"
1,marinara,pizza famiglia,8.6,"[pomodoro, origano, aglio, olio]"
2,margherita,pizza,4.0,"[pomodoro, mozzarella]"
3,margherita,pizza famiglia,9.0,"[pomodoro, mozzarella]"
4,romana,pizza,5.0,"[pomodoro, mozzarella, acciughe, origano]"
...,...,...,...,...
185,sole,pizza famiglia,16.0,"[pomodoro, mozzarella, asparagi, grana, uovo]"
186,napoletana,pizza fritta,6.5,"[scamorza, friarielli]"
187,pizza fritta,pizza fritta,6.5,[]
188,leggera,pizza fritta,6.5,"[philadelfia, pomodorini, mozzarella]"


In [37]:
unsafe_categories = ['pizza', 'pizza famiglia']
kebab_products = ['kebab panino', 'kebab piadina', 'kebab panino menu', 'kebab piadina menu', 'piatto piccolo',
                 'piatto grande', 'pizza kebab', 'piatto falafel', 'panino pizza']
sold_products = []
orders_count = []
orders_total = []
for order_file in os.listdir(orders_dir):
    all_orders = pd.read_json(orders_dir + "/" + order_file)

    orders_count.append([order_file.replace(".json", ""), len(all_orders)])

    orders = all_orders['orders']
    for order in orders:
        order_name = order['name']
#         print(order)
        order_amount = float(order['actualTotal'])
        order_extra_price = float(order['extraPrice'])
        order_type = order['type']
        if order_type != 'DELIVERY':
            # compute the total manually
            order_products = order['products']
            total = 0
            for product in order_products:
                product_name = product['name']
                product_category = product['category']

                like_category = product_category
                if like_category == "pizza famiglia 2 gusti" or like_category == "pizza famiglia 3 gusti" or \
                like_category == "pizza famiglia 4 gusti":
                    like_category = "pizza famiglia"

                product_row = products_df[(products_df['name'] == product_name) &
                                      (products_df['category'] == like_category)]
                if len(product_row) == 0:
                    print(order_file, order_name, product_name, len(product_row))

                product_ingredients = []
                personalized = False
                if product_name not in kebab_products:
                    for ingredient in product['ingredients']:
                        product_ingredients.append(ingredient)
                        if product_category in unsafe_categories:
                            total += 1
                            personalized = True
                    for ingredients in product_row['ingredients']:
                        for ingredient in ingredients:
                            if ingredient not in product_ingredients:
                                product_ingredients.append(ingredient)
                else:
                    if "tutto" in product['ingredients']:
                        for ingredients in product_row['ingredients']:
                            for ingredient in ingredients:
                                if ingredient not in product_ingredients:
                                    product_ingredients.append(ingredient)
                        for ingredient in product['ingredients'][1:]:
                            product_ingredients.append(ingredient)
                    else:
                        for ingredient in product['ingredients']:
                            product_ingredients.append(ingredient)

                product_price = float(product_row['price'])
                total += (product_price * int(product['quantity']))
                sold_products.append([order_file.replace(".json", ""), order_name, product_name, product_ingredients,
                                     personalized, list(product_row["category"])[0]])
            orders_total.append([order_file.replace(".json", ""), order_name, total])
        else:
            if order_amount > 0:
                order_products = order['products']
                for product in order_products:
                    product_name = product['name']
                    product_category = product['category']

                    like_category = product_category
                    if like_category == "pizza famiglia 2 gusti" or like_category == "pizza famiglia 3 gusti" or \
                    like_category == "pizza famiglia 4 gusti":
                        like_category = "pizza famiglia"

                    product_row = products_df[(products_df['name'] == product_name) &
                                          (products_df['category'] == like_category)]
                    if len(product_row) == 0:
                        print(order_file, order_name, product_name, len(product_row))

                    product_ingredients = []
                    personalized = False
                    if product_name not in kebab_products:
                        for ingredient in product['ingredients']:
                            product_ingredients.append(ingredient)
                            if product_category in unsafe_categories:
                                personalized = True
                        for ingredients in product_row['ingredients']:
                            for ingredient in ingredients:
                                if ingredient not in product_ingredients:
                                    product_ingredients.append(ingredient)
                    else:
                        if "tutto" in product['ingredients']:
                            for ingredients in product_row['ingredients']:
                                for ingredient in ingredients:
                                    if ingredient not in product_ingredients:
                                        product_ingredients.append(ingredient)
                            for ingredient in product['ingredients'][1:]:
                                product_ingredients.append(ingredient)
                        else:
                            for ingredient in product['ingredients']:
                                product_ingredients.append(ingredient)
                    sold_products.append([order_file.replace(".json", ""), order_name, product_name, product_ingredients,
                                         personalized, list(product_row["category"])[0]])

                orders_total.append([order_file.replace(".json", ""), order_name, order_amount + order_extra_price])
            else:
                # compute the total manually
                order_products = order['products']
                total = 0
                for product in order_products:
                    product_name = product['name']
                    product_category = product['category']

                    like_category = product_category
                    if like_category == "pizza famiglia 2 gusti" or like_category == "pizza famiglia 3 gusti" or \
                    like_category == "pizza famiglia 4 gusti":
                        like_category = "pizza famiglia"

                    product_row = products_df[(products_df['name'] == product_name) &
                                          (products_df['category'] == like_category)]
                    if len(product_row) == 0:
                        print(order_file, order_name, product_name, len(product_row))

                    product_ingredients = []
                    personalized = False
                    if product_name not in kebab_products:
                        for ingredient in product['ingredients']:
                            product_ingredients.append(ingredient)
                            if product_category in unsafe_categories:
                                total += 1
                                personalized = True
                        for ingredients in product_row['ingredients']:
                            for ingredient in ingredients:
                                if ingredient in ingredients:
                                    if ingredient not in product_ingredients:
                                        product_ingredients.append(ingredient)
                    else:
                        if "tutto" in product['ingredients']:
                            for ingredients in product_row['ingredients']:
                                for ingredient in ingredients:
                                    if ingredient not in product_ingredients:
                                        product_ingredients.append(ingredient)
                            for ingredient in product['ingredients'][1:]:
                                product_ingredients.append(ingredient)
                        else:
                            for ingredient in product['ingredients']:
                                product_ingredients.append(ingredient)

                    product_price = float(product_row['price'])
                    total += (product_price * int(product['quantity']))
                    sold_products.append([order_file.replace(".json", ""), order_name, product_name, product_ingredients,
                                         personalized, list(product_row["category"])[0]])

                orders_total.append([order_file.replace(".json", ""), order_name, total])

In [38]:
for sold_product in sold_products:
    for ingredient in sold_product[3]:
        if isinstance(ingredient, list):
            print("list")

In [50]:
orders_count_df = pd.DataFrame(orders_count, columns=["date", "orders_count"])
orders_count_df["date"] = orders_count_df["date"].apply(lambda x: pd.to_datetime(x, dayfirst=True))
orders_count_df.sort_values(by="date", inplace=True)
orders_count_df

,date,orders_count
0,2022-08-10,73
1,2022-08-14,93
2,2022-08-15,106
3,2022-08-16,56
5,2022-08-19,76
6,2022-08-21,116
7,2022-08-29,53
8,2022-08-30,68
4,2022-09-18,90


In [49]:
orders_total_df = pd.DataFrame(orders_total, columns=["date", "order_name", "total"])
orders_total_df["date"] = orders_total_df["date"].apply(lambda x: pd.to_datetime(x, dayfirst=True))
orders_total_df.sort_values(by="date", inplace=True)
orders_total_df

,date,order_name,total
0,2022-08-10,order1,14.0
52,2022-08-10,order54,6.5
51,2022-08-10,order53,14.5
50,2022-08-10,order52,8.0
49,2022-08-10,order51,6.5
...,...,...,...
390,2022-09-18,order65,8.0
391,2022-09-18,order66,13.5
392,2022-09-18,order67,17.0
394,2022-09-18,order69,23.5


In [41]:
all_ingredients = {}
for sold_product in sold_products:
    for ingredient in sold_product[3]:
        if ingredient not in all_ingredients:
            all_ingredients[ingredient] = 0
        all_ingredients[ingredient] += 1
all_ingredients_count = []
for ingredient_name in all_ingredients:
    all_ingredients_count.append([ingredient_name, all_ingredients[ingredient_name]])
all_ingredients_df = pd.DataFrame(all_ingredients_count, columns=["ingredient", "count"])
all_ingredients_df.sort_values(by="count", ascending=False, inplace=True)
all_ingredients_df

,ingredient,count
6,pomodoro,912
4,mozzarella,640
8,insalata,437
1,patatine,400
12,yogurt,344
...,...,...
21,-rucola,1
74,-grana,1
37,-cappuccio,1
59,-salame piccante,1


In [42]:
all_products = {}
for sold_product in sold_products:
    product_name = sold_product[2]
    if product_name not in all_products:
        all_products[product_name] = 0
    all_products[product_name] += 1
all_products_count = []
for product_name in all_products:
    all_products_count.append([product_name, all_products[product_name]])
all_products_df = pd.DataFrame(all_products_count, columns=["product_name", "count"])
all_products_df.sort_values(by="count", ascending=False, inplace=True)
all_products_df

,product_name,count
5,kebab panino,314
2,kebab piadina,269
3,margherita,227
0,pizza kebab,120
4,coca cola,99
...,...,...
102,seba,1
103,affumicata,1
34,mediterranea,1
105,chef,1


In [43]:
all_categories = {}
for sold_product in sold_products:
    product_category = sold_product[5]
#     print(product_category)
    if product_category not in all_categories:
        all_categories[product_category] = 0
    all_categories[product_category] += 1
all_products_count = []
for product_category in all_categories:
    all_products_count.append([product_category, all_categories[product_category]])
all_products_df = pd.DataFrame(all_products_count, columns=["product_category", "count"])
all_products_df.sort_values(by="count", ascending=False, inplace=True)
all_products_df

,product_category,count
2,kebab,741
3,pizza,639
4,bibita,262
7,patatine,125
0,pizza famiglia,76
6,birra,61
1,panino pizza,21
5,piadina,10
8,pizza fritta,9
